In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scripts.data_import import data_import
from scripts.run_optimization_model import run_model

# Read Data

In [ ]:
file = 's1'
path = 'data/full/data_' + file + '.xlsx'
data, basis, bess, service, revenue_change = data_import(path)
bess['s_eff'] = 1


/Users/angelmah/Documents/codes/bess-optimizer/scripts/data_import.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['basis']  = data['basis'].dropna().drop(['Tag']).replace({'YES': 1, 'NO': 0})
/Users/angelmah/Documents/codes/bess-optimizer/scripts/data_import.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['bess']  = data['bess'].dropna().drop(['Tag', np.NaN]).replace({'YES': 1, 'NO': 0})


In [3]:
# bess['cap_power'] = 4
# bess['cap_energy'] = 16
# bess['s_eff'] = 1

# data['soc_limit']['min'] = 0.1
# data['activation']['reg'] = 0.15
# data['activation']['reg'] = -0.15

# data['activation']['reg_up'] = data['activation']['reg'].apply(lambda x: x if x > 0 else 0)
# data['activation']['reg_down'] = data['activation']['reg'].apply(lambda x: -x if x < 0 else 0)

# # data['activation']['reg_up_activated'] = 1
# # data['activation']['reg_down_activated'] = 1

# data['activation']['reg_up_activated'] = data['activation']['reg'].apply(lambda x: 1 if x > 0 else 0)
# data['activation']['reg_down_activated'] = data['activation']['reg'].apply(lambda x: 1 if x < 0 else 0)


# result_html_name = "result/" + file +"_mipgap0.005" + "_p" + str(bess['cap_power']) + "_e" + str(bess['cap_energy']) + "_l2__"



# RUN

In [3]:
power_range = list(range(1, 5))
energy_range = list(range(1, 5))

econs_sens_keys = []
econs_sens_values = []

for power in power_range:
    for energy in energy_range:

        power_target = power * 0.5
        energy_target = energy * 0.5

        print("Solving: ", power_target, energy_target)

        bess['cap_power'] = power_target
        bess['cap_energy'] = energy_target

        packaged_data = [data, basis, bess, service] 

        annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_name = [])

        [NPV, ROI, IRR, BCR, PP] = financial_metrics

        econs_sens_keys.append([power_target, energy_target])
        econs_sens_values.append([annual_revenue] + financial_metrics)


Solving:  0.5 0.5
Solving:  0.5 1.0
Solving:  0.5 1.5
Solving:  0.5 2.0
Solving:  1.0 0.5
Solving:  1.0 1.0
Solving:  1.0 1.5
Solving:  1.0 2.0
Solving:  1.5 0.5
Solving:  1.5 1.0
Solving:  1.5 1.5
Solving:  1.5 2.0
Solving:  2.0 0.5
Solving:  2.0 1.0
Solving:  2.0 1.5
Solving:  2.0 2.0


In [4]:
df = pd.DataFrame(index = pd.MultiIndex.from_tuples(econs_sens_keys, names = ['Power', 'Energy']), data = econs_sens_values, columns = ['annual_revenue', 'NPV', 'ROI', 'IRR', 'BCR', 'PP'])
df['annual_revenue_per_kw'] = df['annual_revenue']/df.index.get_level_values(0)/1000
df = df.reset_index()
# df['Energy'] = (df['Energy'] * 0.5) * df['Power']
# df = df.set_index(['Power', 'Energy'])

In [5]:
pd.set_option('display.max_rows', None)
df

,Power,Energy,annual_revenue,NPV,ROI,IRR,BCR,PP,annual_revenue_per_kw
0,0.5,0.5,191975.323777,1.086099e+06,2.740709,0.496856,3.740709,3,383.950648
1,0.5,1.0,270318.155863,1.420651e+06,2.130954,0.400852,3.130954,3,540.636312
2,0.5,1.5,318968.686724,1.525928e+06,1.628414,0.324836,2.628414,4,637.937373
3,0.5,2.0,355260.145925,1.535771e+06,1.271908,0.270140,2.271908,4,710.520292
4,1.0,0.5,258129.177224,1.471027e+06,2.817101,0.520760,3.817101,3,258.129177
5,1.0,1.0,383935.393626,2.172080e+06,2.740560,0.496834,3.740560,3,383.935394
6,1.0,1.5,474067.181807,2.597663e+06,2.443807,0.448687,3.443807,3,474.067182
7,1.0,2.0,540675.986938,2.841609e+06,2.131184,0.400885,3.131184,3,540.675987
8,1.5,0.5,283213.228388,1.538826e+06,2.374469,0.462194,3.374469,3,188.808819
9,1.5,1.0,466483.752210,2.683602e+06,2.921845,0.530286,3.921845,2,310.989168


In [6]:
pivot_df = df.pivot(index="Energy", columns="Power", values="annual_revenue_per_kw") * 100
pivot_df

Power,0.5,1.0,1.5,2.0
Energy,,,,
0.5,38395.064755,25812.917722,18880.881893,14826.594134
1.0,54063.631173,38393.539363,31098.916814,25812.919975
1.5,63793.737345,47406.718181,38393.625385,33182.316754
2.0,71052.029185,54067.598694,44707.410325,38387.012203


In [7]:
import plotly.graph_objects as go

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  
    colorbar=dict(title='')
))

# Adding titles and labels
fig.update_layout(
    # title='Heatmap of ROI with varying Power and Energy (Reversed Energy Axis)',
    xaxis=dict(title='Power'),
    yaxis=dict(title='Energy')
)

# Display the figure
fig.show()


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculating Energy/Power ratio for the contour plot
df['Ratio'] = df['Energy'] / df['Power']
pivot_ratio = df.pivot(index="Energy", columns="Power", values="Ratio")

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap with contour lines
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  # High values red, low values blue
    colorbar=dict(title='ROI (%)'),
    zmin=np.min(z),  # Define the range of z for better contrast
    zmax=np.max(z)
))

# Add contour lines for the Energy/Power ratio with text labels
fig.add_trace(go.Contour(
    z=pivot_ratio.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    showscale=False,  # Hide the scale for contour lines
    contours=dict(
        coloring='lines',
        showlabels=True,  # Enable labels on contours
        labelfont=dict(size=12, color='black'),  # Customize the font of the labels
        start=0.5,  # Starting value for contour lines
        end=4,     # Ending value for contour lines
        size=0.5   # Interval between contour lines
    ),
    line=dict(color='black', width=2),
    # ncontours=50  # Increase the number of contour lines

))

# Adding titles and labels, and setting axis ranges to start from 1
fig.update_layout(
    title='Heatmap of ROI with Energy/Power Ratio Contour Lines',
    xaxis=dict(title='Power', range=[0.5, 8]),
    yaxis=dict(title='Energy', range=[0.5, 8])
)

# Display the figure
fig.show()
